## 循环神经网络的程序示例

### 1. RNN 架构

### 2. 实现循环神经网络

- 利用tensorflow实现的循环神经网络RNN（本程序使用了LSTM）来做语言模型，并输出其困惑度。
- 语言模型主要是根据一段给定的文本来预测下一个词最有可能是什么。困惑度用于评价语言模型。困惑度越小，则模型的性能越好

In [5]:
import reader
import numpy as np
import tensorflow as tf

print("tensorflow version : ", tf.__version__)

tensorflow version :  2.3.0


#### 2.1 定义参数

- data目录中应该预先存放本程序中要用到的语料，即PTB(Penn Treebank Dataset)数据集
- PTB数据集是语言模型研究常用的数据集。其下载地址在 Tomas Mikolov的主页：
- 下载地址 : https://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz。 (推荐使用迅雷下载)
- 解压后，将其中的整个data文件夹拷贝到当前目录即可。数据集共有9998个单词，加上稀有词语的特殊符号<unk>和语句的结束标记，共有10000个单词。

In [6]:
# 数据路径
DATA_PATH = './data'

# 超参数的设置
# size of RNN hidden state, 每个单词词向量的维度
HIDDEN_SIZE = 200
# LSTM 层数
NUM_LAYERS = 2
# 词汇表中词的个数
VOCAB_SIZE = 10000
# 学习速率的超参数
LEARNING_RATE = 1.0
# 训练阶段每个数据批量设置为多少个样本, 本例中指若干个词构成的序列
TRAIN_BATCH_SIZE = 20
# 训练阶段文本数据的截断长度, 也可以成为序列长度 seq_length
TRAIN_NUM_STEP = 35
# 训练的轮数
NUM_EPOCH = 2
# 节点不被 dropout 的概率
KEEP_PROB = 0.5

# 超参数, 避免梯度膨胀
MAX_GRAD_NORM = 5

### 2.2 定义多层循环神经网络

- tf.compat.v1.get_variable
    1. 该函数的作用是创建新的tensorflow变量
    2. name :  变量名
    3. shape : []

- tf.nn.dropout()
    1. 参考 [详解tf.nn.dropout](https://blog.csdn.net/yangfengling1023/article/details/82911306)
    2. 该函数的作用是tensorflow里面为了防止或减轻过拟合而使用的函数，它一般用在全连接层
    3. Dropout就是在不同的训练过程中随机扔掉一部分神经元。也就是让某个神经元的激活值以一定的概率p，让其停止工作，这次训练过程中不更新权值，也不参加神经网络的计算。但是它的权重得保留下来（只是暂时不更新而已），因为下次样本输入时它可能又得工作了
- tf.variable_scope()
    1. 参考 [详解tf.variable_scope函数](https://docs.pythontab.com/tensorflow/how_tos/variable_scope/)
    2. 用于定义创建变量（层）的操作的上下文管理器。此上下文管理器验证（可选）values是否来自同一图形，确保图形是默认的图形，并推送名称范围和变量范围

In [8]:
class PTBModel(object):
    def __init__(self, is_training, batch_size, num_steps):
        """
        初始化参数
        :param is_training: 当前是否在训练阶段
        :param batch_size: 批量的大小
        :param num_steps: 数据的截断长度, 即序列长度
        """
        # 定义输入层的数据维度为 batch_size * num_steps
        self.input_data = tf.keras.Input(dtype=tf.int32, shape=[batch_size, num_steps])
        # 定义输出层的数据维度为 batch_size * num_steps
        self.targets = tf.keras.Input(dtype=tf.int32, shape=[batch_size, num_steps])

        # 定义使用LSTM机构作为循环体的基本结构, 每个词向量的维度为 HIDDEN_SIZE
        lstm_cell = tf.compat.v1.nn.rnn_cell.LSTMCell()

        #如果是在训练阶段，则使用dropout.此时每个单元以（1-keep_prob）的概率不工作，目的是防止过拟合。
        if is_training:
            lstm_cell = tf.compat.v1.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob=KEEP_PROB)

        # 将多层RNN单元封装到一个单元Cell中, 层的个数NUM_LAYERS前面已经确定
        cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([lstm_cell] * NUM_LAYERS)

        # 使用zero_state函数初始化网络状态
        self.initial_state = cell.zero_state(batch_size, tf.int32)

        # 将单词转换为词向量
        # VOCAB_SIZE 词的总数, HIDDEN_SIZE 是每个单词的向量维度
        # embedding 的参数为 : VOCAB_SIZE * HIDDEN_SIZE
        # 则input的输入维度为 : batch_size * num_steps * HIDDEN_SIZE
        embedding = tf.compat.v1.get_variable('embedding', [VOCAB_SIZE, HIDDEN_SIZE])
        inputs = tf.nn.embedding_lookup(embedding, self.input_data)

        # 如果在训练阶段, 将进行 dropout
        if is_training:
            inputs = tf.nn.dropout(inputs, KEEP_PROB)

        # 定义LSTM结构的输出列表
        outputs = []
        # 使用state存储LSTM的初始状态
        state = self.initial_state
        #TensorFlow提供了Variable Scope 机制，用于共享变量
        with tf.compat.v1.variable_scope("RNN"):
            # 对于每个时间同步
            for time_step in range(num_steps):
                if time_step > 0:
                    # 重用变量
                    tf.compat.v1.get_variable_scope().reuse_variables()
                # 从输入数据inputs中获取当前时刻的输入并传入LSTM的单元
                # 每次输出都是一个张量, shape = cell([20, 200]), 其中20是BATCH_SIZE, 200是词向量维度
                cell_output, state = cell(inputs[:, time_step, :], state)
                # 将当前的单元的输出加入到输出的列表中
                outputs.append(cell_output)

        # 将输出的列表 outputs 利用tf.concat函数变成（batch,hidden_size*num_steps）的形状，然后再reshape成（batch*num_steps,hidden_size）的形状
        # 即为 （20*35， 200）=（700, 200）
        output = tf.resource(tf.concat(outputs, 1), [-1, HIDDEN_SIZE])

        # 利用刚才LSTM的输出向量output乘以权重weight再加上偏置bias，得到最后的预测结果logits
        # 获取权重值
        weight = tf.compat.v1.get_variable('weight', [HIDDEN_SIZE, VOCAB_SIZE])  # weight 的形状为 [200,10000]
        # 获取偏置值
        bias = tf.compat.v1.get_variable('bias', [VOCAB_SIZE])
        # 获取预测结果
        logits = tf.matmul(output, weight) + bias

        # 计算交叉熵损失
        loss = tf.keras.losses.categorical_crossentropy(
            [logits]  # 预测结果 shape=(700, 10000)
            [tf.reshape(self.targets, [-1])]  # self.targets是正确的结果.这里对其shape进行了调整，变为shape=(700,)
        )

        # 计算每个 batch 的平均损失值
        self.cost = tf.reduce_sum(loss) / batch_size
        # 更新网络状态
        self.final_state = state

        # 如果当前是在训练阶段, 则进行下面的反向传播以更新梯度
        if not is_training:
            return

        # 返回需要训练的变量
        trainable_variables = tf.compat.v1.trainable_variables()

        # 通过clip_by_global_norm函数控制梯度的大小，避免梯度膨胀问题
        grads, _ = tf.clip_by_global_norm(
            tf.gradients(self.cost, trainable_variables), MAX_GRAD_NORM
        )

        # 定义模型的优化方法
        optimizer = tf.compat.v1.train.GradientDescentOptimizer(LEARNING_RATE)
        # 应用梯度对 trainable_variables 更新
        self.train_op = optimizer.apply_gradients(zip(grads, trainable_variables))


In [ ]:
# 定义的函数run_epoch。使用刚才定义的模型model在数据data上运行train_op并返回perplexity值

def run()

#### 2.3 定义主函数

In [12]:
from tensorflow.models.tutorials.rnn.ptb import  reader

AttributeError: module 'tensorflow' has no attribute 'flags'